In [109]:
import glob
import pandas as pd
import numpy as np
# import scipy.sparse 
from sklearn.preprocessing import normalize
from sklearn.svm import SVC
from sklearn.metrics.pairwise import chi2_kernel
from sklearn.cluster import KMeans
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import average_precision_score
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import pickle
import time
import os
import sys

In [115]:
os.chdir('/home/ubuntu/hw1_git/11775-hws/hw1_code')

In [114]:
os.getcwd()

'/home/ubuntu/hw1_git/11775-hws/hw1_code'

In [104]:
#python2 scripts/train_svm.py $event "kmeans/" $feat_dim_mfcc mfcc_pred/svm.$event.model || exit 1;
sys.argv = ['cripts/train_svm.py', 'P001', 'kmeans/', '3', 'mfcc_pred/P001.model', 'mfcc']

In [105]:
if __name__ == '__main__':
    if len(sys.argv) != 6:
        print "Usage: {0} event_name feat_dir feat_dim output_file".format(sys.argv[0])
        print "event_name -- name of the event (P001, P002 or P003 in Homework 1)"
        print "feat_dir -- dir of feature files"
        print "feat_dim -- dim of features"
        print "output_file -- path to save the svm model"
        print "model_type -- mfcc or asrs"
        exit(1)

    event_name = sys.argv[1]
    feat_dir = sys.argv[2]
    feat_dim = int(sys.argv[3])
    output_file = sys.argv[4]
    model_type = sys.argv[5]
    print event_name
    print feat_dir
    print feat_dim
    print output_file
    print model_type


    print 'SVM trained successfully for event %s!' % (event_name)

P001
kmeans/
3
mfcc_pred/P001.model
mfcc
SVM trained successfully for event P001!


In [52]:
if model_type == 'mfcc':
    train_mart = pd.read_csv('./datamart_train_k{}.csv'.format(feat_dim))
    valid_mart = pd.read_csv('./datamart_valid_k{}.csv'.format(feat_dim))
    test_mart  = pd.read_csv('./datamart_test_k{}.csv'.format(feat_dim))
    print(train_mart.shape, valid_mart.shape, test_mart.shape)
    LGBM_results_p001 = modeling_ap_lgbm(k=feat_dim, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
    Xgb_results_p002 = modeling_ap_xgb(k=feat_dim, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
    Xgb_results_p003 = modeling_ap_xgb(k=feat_dim, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')
elif model_type == 'asrs':
    train_mart = pd.read_csv('./datamart_train_asrfeat_{}.csv'.format(feat_dim))
    valid_mart = pd.read_csv('./datamart_valid_asrfeat_{}.csv'.format(feat_dim))
    test_mart  = pd.read_csv('./datamart_test_asrfeat_{}.csv'.format(feat_dim))
    print(train_mart.shape, valid_mart.shape, test_mart.shape)
    Boost_results_p001 = modeling_ap_Boost(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
    LGBM_results_p002 = modeling_ap_lgbm(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
    SVM_results_p003 = modeling_ap_SVM(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

((836, 12), (400, 12), (1699, 12))


In [43]:
# print(train_mart.dtypes, valid_mart.dtypes, test_mart.dtypes)
print(train_mart.shape, valid_mart.shape, test_mart.shape)

((836, 12), (400, 12), (1699, 12))


In [56]:
def modeling_ap_SVM(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = SVC(kernel=chi2_kernel, probability=True)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of SVM Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs, model

def modeling_ap_AdaB(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = AdaBoostClassifier(n_estimators=200, random_state=0)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of AdaBoosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs, model

def modeling_ap_Boost(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = GradientBoostingClassifier(n_estimators=200, random_state=0)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of Boosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs, model

def modeling_ap_xgb(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = XGBClassifier()
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of XgBoosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs, model

def modeling_ap_lgbm(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = LGBMClassifier(random_state=0, n_jobs=-1)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of XgBoosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs, model

In [83]:
precision, y_probs, model = modeling_ap_lgbm(k=feat_dim, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
# Xgb_results_p002 = modeling_ap_xgb(k=feat_dim, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
# Xgb_results_p003 = modeling_ap_xgb(k=feat_dim, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

===== The time consuming of XgBoosting Modeling : 0.0810928344727 seconds =====
0.21152819198165373


In [110]:
foo = pd.Series({'Validation-AP':precision})
foo_y = pd.DataFrame(y_probs)
foo_y.columns = ['N', 'Y']
foo_y2 = pd.concat([valid_mart, foo_y], axis = 1)[['video', 'N', 'Y']]
foo.to_csv('./mfcc_pred/{}_k{}_model_Val_AP.csv'.format(event_name, feat_dim), index = False)
foo_y2.to_csv('./mfcc_pred/{}_k{}_model_Val_Scores.csv'.format(event_name, feat_dim), index = False)
# save the model to disk
filename = output_file+'.sav'
pickle.dump(model, open(filename, 'wb'))

cripts/train_svm.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [97]:
pickle.dump(foo, open(filename, 'wb'))

,0,1
0,0.421026,0.578974
1,0.725334,0.274666
2,0.999710,0.000290
3,0.999982,0.000018
4,0.999997,0.000003
5,0.796739,0.203261
6,0.999913,0.000087
7,0.999990,0.000010
8,0.999997,0.000003
9,0.999995,0.000005


## test_svm.py

In [145]:
#python2 scripts/test_svm.py mfcc_pred/$event.model "kmeans/" $feat_dim_mfcc mfcc_pred/${event}_mfcc.lst || exit 1;
sys.argv = ['scripts/test_svm.py', 'P001', 'kmeans/', '100', 'mfcc_pred/P001_mfcc.list', 'mfcc']

In [146]:
if __name__ == '__main__':
    if len(sys.argv) != 6:
        print "Usage: {0} model_file feat_dir feat_dim output_file".format(sys.argv[0])
        print "model_file -- path of the trained svm file"
        print "feat_dir -- dir of feature files"
        print "feat_dim -- dim of features; provided just for debugging"
        print "output_file -- path to save the prediction score"
        print "model_type -- mfcc or asrs"
        exit(1)

    model_file = sys.argv[1]
    feat_dir = sys.argv[2]
    feat_dim = int(sys.argv[3])
    output_file = sys.argv[4]
    model_type = sys.argv[5]
    print model_file
    print feat_dir
    print feat_dim
    print output_file
    print model_type

P001
kmeans/
100
mfcc_pred/P001_mfcc.list
mfcc


In [122]:
feat_dim = 100

In [ ]:
if model_type == 'mfcc':
    test_mart  = pd.read_csv('./datamart_test_k{}.csv'.format(feat_dim))
    print('test datamart shape is', test_mart.shape)
elif model_type == 'asrs':
    test_mart  = pd.read_csv('./datamart_test_asrfeat_{}.csv'.format(feat_dim))
    print('test datamart shape is', test_mart.shape)

In [116]:
os.getcwd()

'/home/ubuntu/hw1_git/11775-hws/hw1_code'

In [153]:
# some time later...
# load the model from disk
import pickle
filename = 'mfcc_pred/'+model_file+'.model.sav'
model = pickle.load(open(filename, 'rb'))
print(model)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


In [125]:
test_mart  = pd.read_csv('./datamart_test_k{}.csv'.format(feat_dim))
X_test = test_mart.iloc[:,1:feat_dim+1]
X_test.head(3)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99
0,0.005898,0.078637,0.004587,0.0,0.000000,0.000000,0.134338,0.000000,0.000000,0.005242,...,0.000655,0.000655,0.000000,0.001311,0.000000,0.079292,0.001966,0.060288,0.00000,0.000000
1,0.000000,0.000000,0.034764,0.0,0.016445,0.039541,0.000000,0.002781,0.000242,0.000000,...,0.000000,0.003204,0.000726,0.000000,0.000121,0.002177,0.000846,0.000423,0.00006,0.046010
2,0.000000,0.000000,0.000615,0.0,0.027730,0.011190,0.000000,0.000061,0.002582,0.000000,...,0.000000,0.000000,0.000676,0.000000,0.000061,0.000000,0.000000,0.000000,0.00000,0.006333


In [151]:
scores = model.predict_proba(X_test)
scores_df = pd.DataFrame(scores)
scores_df.columns = ['N', 'Y']
scores_total = pd.concat([test_mart, scores_df], axis = 1)
scores_total.head(3)

,video,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,target,Data,target_p001,target_p002,target_p003,target_p001_10,target_p002_10,target_p003_10,N,Y
0,HVC1037,0.005898,0.078637,0.004587,0.0,0.000000,0.000000,0.134338,0.000000,0.000000,...,UNK,TEST,Other,Other,Other,0,0,0,0.999763,0.000237
1,HVC4219,0.000000,0.000000,0.034764,0.0,0.016445,0.039541,0.000000,0.002781,0.000242,...,UNK,TEST,Other,Other,Other,0,0,0,0.999956,0.000044
2,HVC4734,0.000000,0.000000,0.000615,0.0,0.027730,0.011190,0.000000,0.000061,0.002582,...,UNK,TEST,Other,Other,Other,0,0,0,0.999899,0.000101


In [139]:
test_list = pd.read_csv('./list/test.video', header = None)
test_list.columns = ['video']
test_list.head(3)

,video
0,HVC286
1,HVC288
2,HVC314


In [158]:
final_list = test_list.merge(scores_total, how = 'left', on = 'video')[['video', 'Y']]
final_list.to_csv(output_file+'.csv', index = False)
final_list['Y'].to_csv(output_file+'.score'+'.csv', index = False, header = True)

In [154]:
final_list['Y']

0       0.000015
1       0.000008
2       0.002519
3       0.000004
4       0.000006
5       0.000070
6       0.000182
7       0.000052
8       0.000037
9       0.000026
10      0.000005
11      0.000009
12      0.000015
13      0.000067
14      0.000008
15      0.000008
16      0.000023
17      0.000052
18      0.001352
19      0.000093
20      0.000034
21      0.000012
22      0.000006
23      0.001198
24      0.000003
25      0.000003
26      0.000003
27      0.000005
28      0.000026
29      0.000022
          ...   
1669    0.000020
1670    0.000432
1671    0.000002
1672    0.449533
1673    0.000145
1674    0.000097
1675    0.000004
1676    0.000009
1677    0.449533
1678    0.003115
1679    0.000005
1680    0.000112
1681    0.000591
1682    0.000167
1683    0.007150
1684    0.000099
1685    0.000785
1686    0.003290
1687    0.000249
1688    0.000107
1689    0.000134
1690    0.167472
1691    0.000015
1692    0.000697
1693    0.000004
1694    0.000011
1695    0.000004
1696    0.0000